In [1]:
import os
from IPython.display import display
from typing import Union

import torch as T
import torch.nn as nn
import torch.optim as optim

import torchinfo

from admon.model import DMoN
from admon.utils import load_npz, normalize_graph, modularity

# Alias
_PathLike = Union[str, 'os.PathLike[str]']
CORA_DIR: _PathLike = './data/cora'

In [3]:
adj, features, labels, label_indices = load_npz(os.path.join(CORA_DIR, 'cora.npz'))
adj_tensor = T.tensor(adj.todense()).unsqueeze(0).float()
features_tensor = T.tensor(features.todense()).unsqueeze(0).float()

In [6]:
# hyperparameter
n_clusters: int = 16
num_epochs: int = 200
hidden: int = 64
depths: int = 1
dropout: float = 0.3
inflation: int = 1
collapse_regularization = 1
device = T.device('cpu')

lr: float = 1e-3
weight_decay: float = 5e-4
lr_decay_step: int = 5
lr_decay_gamma: float = 0.3

model = DMoN(features_tensor.size(-1), n_clusters,
             hidden, depths, dropout, inflation,
             collapse_regularization=collapse_regularization)
model: nn.Module = model.to(device)
torchinfo.summary(model)

Layer (type:depth-idx)                   Param #
DMoN                                     --
├─Single: 1-1                            --
│    └─Sequential: 2-1                   --
│    │    └─GCN: 3-1                     91,712
│    └─Linear: 2-2                       1,040
│    └─Dropout: 2-3                      --
Total params: 92,752
Trainable params: 92,752
Non-trainable params: 0

In [7]:
# Train main
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, lr_decay_step, lr_decay_gamma)

for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()

  pooled_features, assignments = model((features_tensor, adj_tensor))
  model.loss.backward()
  optimizer.step()
  lr_scheduler.step()

  print(f'Epoch [{epoch+1:d}/{num_epochs:d}] Loss: {model.loss.item():.2f}')

Epoch [1/200] Loss: 2707.08
Epoch [2/200] Loss: 2707.05
Epoch [3/200] Loss: 2707.06
Epoch [4/200] Loss: 2707.03
Epoch [5/200] Loss: 2707.00
Epoch [6/200] Loss: 2707.00
Epoch [7/200] Loss: 2707.01
Epoch [8/200] Loss: 2707.01
Epoch [9/200] Loss: 2707.00
Epoch [10/200] Loss: 2707.00
Epoch [11/200] Loss: 2706.99
Epoch [12/200] Loss: 2706.99
Epoch [13/200] Loss: 2706.99
Epoch [14/200] Loss: 2706.99
Epoch [15/200] Loss: 2706.99
Epoch [16/200] Loss: 2706.99
Epoch [17/200] Loss: 2706.99
Epoch [18/200] Loss: 2706.98
Epoch [19/200] Loss: 2706.98
Epoch [20/200] Loss: 2706.98
Epoch [21/200] Loss: 2706.98
Epoch [22/200] Loss: 2706.98
Epoch [23/200] Loss: 2706.98
Epoch [24/200] Loss: 2706.98
Epoch [25/200] Loss: 2706.98
Epoch [26/200] Loss: 2706.98
Epoch [27/200] Loss: 2706.98
Epoch [28/200] Loss: 2706.98
Epoch [29/200] Loss: 2706.98
Epoch [30/200] Loss: 2706.98
Epoch [31/200] Loss: 2706.98
Epoch [32/200] Loss: 2706.98
Epoch [33/200] Loss: 2706.98
Epoch [34/200] Loss: 2706.98
Epoch [35/200] Loss: 27

In [ ]:
assignments.detach().cpu().numpy().argmax(axis=-1)


array([[0, 9, 7, ..., 9, 9, 9]])

In [ ]:
T.sparse.spmm(adj_tensor, features_tensor)

AttributeError: module 'torch.sparse' has no attribute 'spmm'